# Nulls, Errors, and Exceptions - The Scala Way

These three concepts are amongst the most difficult to deal with in any Java program.  They are the source of many kinds of bugs.  But Scala has some tools to help deal with them, reduce boilerplate code, increasing overall quality, and reduce those bugs related to them.

### ➤ * Nulls *

If you have any function, can you tell from it's type signature if it might return a null?  Do you need to write code and tests to check for a null return value?  They are probably the single biggest source of runtime exceptions, because developers do not realize a function might return a null or forget to check for it.

In Scala, we try to avoid nulls completely.  We have an alternative, the __[Option](https://www.scala-lang.org/api/2.12.8/scala/Option.html)__ type (side note - Java 8 borrowed this concept for it's `Optional` type).  Of the three types we will be discussing, this one is the simplest.  It has a type signature that takes one parameter and looks like `Option[A]`.  A null is represented by a `None[A]`, which holds zero elements of type A.  Any non-null value is represented by a `Some[A]`, which holds one element of type A, namely the value.

In [1]:
val option1: Option[String] = None
val option2: Option[String] = Some("foobar")

option1: Option[String] = None
option2: Option[String] = Some("foobar")

An option can also be constructed by in the following way, based on a variable that may or may not be null.  This is useful for wrapping functions which are known to possibly return a null value.

In [2]:
val optionVar1: String = null
val optionVar2: String = "foobar"

Option(optionVar1)
Option(optionVar2)

optionVar1: String = null
optionVar2: String = "foobar"
res1_2: Option[String] = None
res1_3: Option[String] = Some("foobar")

You can check to see if an `Option` is a `None` or a `Some` with the `isEmpty` or `isDefined` method.

In [3]:
option1.isEmpty
option2.isEmpty
option1.isDefined
option2.isDefined

res2_0: Boolean = true
res2_1: Boolean = false
res2_2: Boolean = false
res2_3: Boolean = true

That in turn can be used with `if` conditional to take action based on the kind of `Option`.  You can use the `get` method to extract the value from a `Some`, but will raise an exception for a `None`.  

In [4]:
def printer1(option: Option[String]): String =
  if (option.isDefined) "value = %s".format(option.get) else "no value present"

printer1(option1)
printer1(option2)

defined function printer1
res3_1: String = "no value present"
res3_2: String = "value = foobar"

A less idiomatic way would be to use the `match` and `case` statements.  This way is though probably more clear to developers coming from Java.  It does use pattern matching and deconstruction.

In [5]:
def printer2(option: Option[String]): String =
  option match {
    case Some(v) => "value = %s".format(v)
    case None => "no value present"
  }

printer2(option1)
printer2(option2)

defined function printer2
res4_1: String = "no value present"
res4_2: String = "value = foobar"

If you have one or more transformations that always return a value, you can chain together calls to the `map` method.  It takes a function with the type signature `A => B`.  Note that in the case of the `None[A]`, the result is a `None[B]`.  The transformation is applied to any value held within the `Option`.

In [6]:
def valueFunction1(x: String): Int = x.size

option1.map(valueFunction1)
option2.map(valueFunction1)

defined function valueFunction1
res5_1: Option[Int] = None
res5_2: Option[Int] = Some(6)

A better alternative to `get` is the `getOrElse` method, which extracts the value from a `Some`, or returns a specified default value for a `None`.  Note that the type of the argument must match the inner type of the `Option`.

We can now combine the `map` and the `getOrElse` to create the most functional form of the printer.

In [7]:
def printer3(option: Option[String]): String =
  option.map("value = %s".format(_)).getOrElse("no value present")

printer3(option1)
printer3(option2)

defined function printer3
res6_1: String = "no value present"
res6_2: String = "value = foobar"

If you have one or more transformations that each return an Option, you can chain together calls to the `flatMap` method.  It takes a function with the type signature `A => Option[B]`.  The "map" part returns an `Option[Option[B]]`, and the "flat" part takes that and returns just a `Option[B]`.  It is identical as calling the `map` method followed by the `flatten` method.

In [8]:
def optionFunction1Pass(x: String): Option[Int] = Some(x.size)
def optionFunction2Pass(x: Int): Option[Int] = Some(x * 3)
def optionFunction3Pass(x: Int): Option[Double] = Some(x.toDouble)
def optionFunction3Fail(x: Int): Option[Double] = None
def optionFunction4Pass(x: Double): Option[String] = Some(x.toString)

defined function optionFunction1Pass
defined function optionFunction2Pass
defined function optionFunction3Pass
defined function optionFunction3Fail
defined function optionFunction4Pass

If all functions return a `Some`, then the final result will be a `Some`.

In [9]:
option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Pass)
  .flatMap(optionFunction4Pass)

res8: Option[String] = Some("18.0")

If any function in the chain returns a `None`, all the subsequent `flatMap` calls will also return a `None`, but the type will change with each transformation.

In [10]:
option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Fail)
  .flatMap(optionFunction4Pass)

res9: Option[String] = None

This is the power of functional programming - composition.  But here we are composing together functions which may (only conceptually now) return nulls.  Without `Option`, for N transformations, you would have to write an N-level deep nested `if` structure, checking for nulls at each level.  This is not readable or easily maintained code.

Of course, `map` and `flatMap` can be mixed together in a sequence of calls where some transformations return a value and some return an `Option`.

There are a number of other useful methods on `Option`, like `contains`, `exists`, `filter`, etc.  Please refer to the documentation linked above.

In [11]:
option1.contains("blah")
option2.contains("blah")
option2.contains("foobar")

res10_0: Boolean = false
res10_1: Boolean = false
res10_2: Boolean = true

In [12]:
option1.exists(_.size == 5)
option2.exists(_.size == 5)
option2.exists(_.size == 6)

res11_0: Boolean = false
res11_1: Boolean = false
res11_2: Boolean = true

In [13]:
option1.filter(_ == "blah")
option2.filter(_ == "blah")
option2.filter(_ == "foobar")

res12_0: Option[String] = None
res12_1: Option[String] = None
res12_2: Option[String] = Some("foobar")

It can also be converted to an `Either`, as a `Left` or `Right` - discussed further below.

In [14]:
case class Thing(id: Int)

option1.toLeft(Thing(1))
option2.toLeft(Thing(2))

option1.toRight(Thing(1))
option2.toRight(Thing(2))

defined class Thing
res13_1: Either[String, Thing] = Right(Thing(1))
res13_2: Either[String, Thing] = Left("foobar")
res13_3: Either[Thing, String] = Left(Thing(1))
res13_4: Either[Thing, String] = Right("foobar")

In can also be converted to a `Try`, as either a `Failure` or `Success` - discussed further below.

In [15]:
import scala.util._

option1.map(Success(_)).getOrElse(Failure(new Exception("no data")))
option2.map(Success(_)).getOrElse(Failure(new Exception("no data")))

import scala.util._


res14_1: Try[String] = Failure(java.lang.Exception: no data)
res14_2: Try[String] = Success("foobar")

### ➤ * Errors *

In Scala, we usually deal with a method that can return a value or some kind of error using the __[Either](https://www.scala-lang.org/api/2.12.8/scala/util/Either.html)__ type.  It has a type signature that takes two parameters and looks like `Either[A, B]`.  A failure is represented by a `Left[A]`. A Success is represented by a `Right[B]`.

In [16]:
case class Error(message: String)

val either1: Either[Error, Int] = Left(Error("error"))
val either2: Either[Error, Int] = Right(123)

defined class Error
either1: Either[Error, Int] = Left(Error("error"))
either2: Either[Error, Int] = Right(123)

Either is right biased, meaning operators like `map` and `flatMap` only operate on a Right.  For example, the type signature for map is `map[C](B => C): Either[A,C]`.  Only the value within a `Right` will be operated on.  The value within a `Left` will be unchanged.  Either way, the type signature of the `Either` will changed.

In [17]:
either1.map(_.toDouble)
either2.map(_.toDouble)

res16_0: Either[Error, Double] = Left(Error("error"))
res16_1: Either[Error, Double] = Right(123.0)

Just like with `Option`, functions which return an `Either` can be chained together using `flatMap`.

In [18]:
def eitherFunc1Pass(x: Int): Either[Error, Int] = Right(x + 321)
def eitherFunc2Pass(x: Int): Either[Error, Int] = Right(x * 2)
def eitherFunc3Pass(x: Int): Either[Error, Double] = Right(x.toDouble)
def eitherFunc3Fail(x: Int): Either[Error, Double] = Left(Error("function 3 failed"))
def eitherFunc4Pass(x: Double): Either[Error, String] = Right(x.toString)

defined function eitherFunc1Pass
defined function eitherFunc2Pass
defined function eitherFunc3Pass
defined function eitherFunc3Fail
defined function eitherFunc4Pass

Here is an example with all `Right`s being returned.

In [19]:
either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Pass)
  .flatMap(eitherFunc4Pass)

res18: Either[Error, String] = Right("888.0")

Here is an example where one function returns a `Left`, as does all subsequent transformations.

In [20]:
either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Fail)
  .flatMap(eitherFunc4Pass)

res19: Either[Error, String] = Left(Error("function 3 failed"))

There are the `boolean` methods `contains` and `exists`, which may be useful.

In [21]:
either1.contains(123)
either2.contains(123)
either2.contains(456)

res20_0: Boolean = false
res20_1: Boolean = true
res20_2: Boolean = false

In [22]:
either1.exists(_ > 111)
either2.exists(_ > 111)
either2.exists(_ > 222)

res21_0: Boolean = false
res21_1: Boolean = true
res21_2: Boolean = false

An `Either[A, B]` can be converted to an `Option[B]`, with a `Left[A]` becoming a `None[B]`, and a `Right[B]` becoming a `Some[B]`.

In [23]:
either1.toOption
either2.toOption

res22_0: Option[Int] = None
res22_1: Option[Int] = Some(123)

### ➤ * Exceptions *

For the situation where a function might throw an exception, we have the `Try` type, which operates very similarly to the `Either` type.  It takes a single type parameter and has the type signature of `Try[A]`. failure is represented by a `Failure`, which actually holds an exception of type `Throwable`.  A success is represented by a `Success[A]` and holds the returned value.

In [24]:
val try1 = Try(3 / 0)
val try2 = Try(3 / 1)

try1: Try[Int] = Failure(java.lang.ArithmeticException: / by zero)
try2: Try[Int] = Success(3)

There are `boolean` methods to check the type `isFailure` and `isSuccess`.

In [25]:
try1.isFailure
try2.isFailure
try1.isSuccess
try2.isSuccess

res24_0: Boolean = true
res24_1: Boolean = false
res24_2: Boolean = false
res24_3: Boolean = true

The same concepts apply here.  A `map` method operates on the value within a `Success`, and only changes the inner type of a `Failure`.

In [26]:
try1.map(_.toString)
try2.map(_.toString)

res25_0: Try[String] = Failure(java.lang.ArithmeticException: / by zero)
res25_1: Try[String] = Success("3")

You can use a `getOrElse`, just like with `Option`, to return some default value in the case of a `Failure`.

In [27]:
try1.map(_.toString).getOrElse("0")
try2.map(_.toString).getOrElse("0")

res26_0: String = "0"
res26_1: String = "3"

A `Try[A]` can be converted to an `Option[A]`, with any `Failure` becoming a `None[A]`.

In [28]:
try1.toOption
try2.toOption

res27_0: Option[Int] = None
res27_1: Option[Int] = Some(3)

A `Try[A]` can be converted to an `Either[Throwable, A]`, with any `Failure` becoming a `Left[Throwable]`.

In [29]:
try1.toEither
try2.toEither

res28_0: Either[Throwable, Int] = Left(java.lang.ArithmeticException: / by zero)
res28_1: Either[Throwable, Int] = Right(3)